In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

import numpy as np
traval_tra_features = np.load('traval_tra_features.npy').astype(np.float32)
traval_tra_targets = np.load('traval_tra_targets.npy').astype(np.int64)[:,0]
traval_val_features = np.load('traval_val_features.npy').astype(np.float32)
traval_val_targets = np.load('traval_val_targets.npy').astype(np.int64)[:,0]
traval_tra_features.shape, traval_val_features.shape

((687636, 2600), (76405, 2600))

In [2]:
import torch
from torch import optim
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetClassifier
import sklearn
from skorch.callbacks import EpochScoring, LRScheduler, Checkpoint
from torch.optim import Adam, SGD
import adamod

/home/amax/miniconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
from skorch.dataset import Dataset
from skorch.helper import predefined_split
valid_ds = Dataset(traval_val_features, traval_val_targets)

In [4]:
class DownSample2x(nn.Sequential):
    def __init__(self, _in, _out):
        super().__init__(
            nn.Conv1d(_in, _out, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
        )

class SELayer(nn.Module):
    def __init__(self, _in, _hidden=64):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(_in, _hidden),
            nn.PReLU(),
            nn.Linear(_hidden, _in),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1)
        return x * y
    
class ResConv1d(nn.Module):
    def __init__(self, _in, _out):
        super(ResConv1d, self).__init__()
        
        self.cal = nn.Sequential(
            nn.Conv1d(_in, _out, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm1d(_out),
            nn.ReLU(),
            nn.Conv1d(_out, _out, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm1d(_out),
        )
        self.se = SELayer(_out, _out)
        self.conv = nn.Conv1d(_in, _out, kernel_size=1, padding=0, stride=1)
        self.relu = nn.ReLU()
        self.bn = nn.BatchNorm1d(_out)
        
    def forward(self, x):
        res = self.cal(x)
        res = self.se(res)
        
        x = self.bn(self.conv(x))
        
        return self.relu(res + x)
        
class MyModule(nn.Module):
    def __init__(self, nonlin=F.relu):
        super(MyModule, self).__init__()
        
        self.d1 = DownSample2x(1, 64)
        self.c1 = ResConv1d(64, 64)
        
        self.d2 = DownSample2x(64, 64)
        self.c2 = ResConv1d(64, 64)
        
        self.d3 = DownSample2x(64, 64)
        self.c3 = ResConv1d(64, 64)
        
        self.d4 = DownSample2x(64, 64)
        self.c4 = ResConv1d(64, 64)
        
        self.d5 = DownSample2x(64, 64)
        self.c5 = ResConv1d(64, 64)
        
        self.d6 = DownSample2x(64, 64)
        self.c6 = ResConv1d(64, 64)
        
        self.dropout = nn.Dropout(0.5)
        self.cls = nn.Linear(64 * 40, 3)
        
    def preprocess(self, x, p=2, eps=1e-8):
        x = x / (x.norm(p=p, dim=1, keepdim=True)+eps)
        x = x.unsqueeze(1)
        return x
        
    def forward(self, x):
        bs = x.size(0)
        x = self.preprocess(x)
        
        x = self.d1(x)
        x = self.c1(x)
        
        x = self.d2(x)
        x = self.c2(x)
        
        x = self.d3(x)
        x = self.c3(x)

        x = self.d4(x)
        x = self.c4(x)
        
        x = self.d5(x)
        x = self.c5(x)
        
        x = self.d6(x)
        x = self.c6(x)
        
        x = x.reshape(bs, -1)
        x = self.dropout(x)

        return F.softmax(self.cls(x))

    
def microf1(net, ds, y=None):
    y_true = [y for _, y in ds]
    y_pred = net.predict(ds)
    return sklearn.metrics.f1_score(y_true, y_pred,average='micro')
def macrof1(net, ds, y=None):
    y_true = [y for _, y in ds]
    y_pred = net.predict(ds)
    return sklearn.metrics.f1_score(y_true, y_pred,average='macro')

class F1Loss(nn.Module):
    def __init__(self):
        super(F1Loss, self).__init__()

    def forward(self, input, target, eps=1e-10):
        loss = 0
        for idx, i in enumerate(torch.eye(3).cuda()):
            t = i.view(3,1)
            y_pred_ = input.matmul(t).squeeze()
            y_true_ = target==idx
            loss += 0.5 * (y_true_ * y_pred_).sum() / (y_true_ + y_pred_ + eps).sum()
        return -torch.log(loss+eps)

## General Training

In [5]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=16,
    lr=0.001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model21')],
    device='cuda'
)

net.fit(traval_tra_features, traval_tra_targets)

/home/amax/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


  epoch    macrof1    microf1    train_loss    valid_acc    valid_loss    cp       dur
-------  ---------  ---------  ------------  -----------  ------------  ----  --------
      1     0.9425     0.9727        0.0338       0.9727        0.0777     +  301.9974
      2     0.9744     0.9940        0.0204       0.9940        0.0197     +  304.2222
      3     0.9657     0.9885        0.0175       0.9885        0.0371        305.2140
      4     0.9755     0.9938        0.0161       0.9938        0.0203     +  306.2678
      5     0.9805     0.9948        0.0150       0.9948        0.0176     +  305.3813
      6     0.9827     0.9958        0.0144       0.9958        0.0144     +  311.1043
      7     0.9808     0.9954        0.0135       0.9954        0.0148        306.7435
      8     0.9826     0.9957        0.0131       0.9957        0.0147        305.9699
      9     0.9822     0.9957        0.0126       0.9957        0.0148        305.2845
     10     0.9817     0.9955        0.0120

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (d1): DownSample2x(
      (0): Conv1d(1, 64, kernel_size=(2,), stride=(2,))
      (1): ReLU(inplace=True)
    )
    (c1): ResConv1d(
      (cal): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool1d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): PReLU(num_parameters=1)
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): Sigmoid()
        )
      )
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (relu): ReLU()
 

## MacroLoss Fintuning

In [7]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=10,
    lr=0.0001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    criterion=F1Loss,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model21', fn_prefix='f1')],
    device='cuda'
)

net.initialize() 
net.load_params(f_params='03_model21/params.pt')
net.partial_fit(traval_tra_features, traval_tra_targets)

/home/amax/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


  epoch    macrof1    microf1    train_loss    valid_acc    valid_loss    cp       dur
-------  ---------  ---------  ------------  -----------  ------------  ----  --------
      1     0.9844     0.9962        0.3043       0.9962        0.3092     +  303.6465
      2     0.9838     0.9961        0.3008       0.9961        0.3092        304.9288
      3     0.9841     0.9962        0.3012       0.9962        0.3093        302.6331
      4     0.9846     0.9962        0.3002       0.9962        0.3088     +  307.6701
      5     0.9840     0.9961        0.2990       0.9961        0.3099        308.2073
      6     0.9837     0.9961        0.2996       0.9961        0.3098        305.5918
      7     0.9843     0.9962        0.2997       0.9962        0.3096        304.3351
      8     0.9842     0.9962        0.2985       0.9962        0.3096        302.8951
      9     0.9841     0.9961        0.2991       0.9961        0.3095        306.0301
     10     0.9843     0.9962        0.2975

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (d1): DownSample2x(
      (0): Conv1d(1, 64, kernel_size=(2,), stride=(2,))
      (1): ReLU(inplace=True)
    )
    (c1): ResConv1d(
      (cal): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool1d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): PReLU(num_parameters=1)
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): Sigmoid()
        )
      )
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (relu): ReLU()
 

In [8]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=16,
    lr=0.001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model22')],
    device='cuda'
)

net.fit(traval_tra_features, traval_tra_targets)

net = NeuralNetClassifier(
    MyModule,
    max_epochs=10,
    lr=0.0001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    criterion=F1Loss,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model22', fn_prefix='f1')],
    device='cuda'
)

net.initialize() 
net.load_params(f_params='03_model22/params.pt')
net.partial_fit(traval_tra_features, traval_tra_targets)

/home/amax/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


  epoch    macrof1    microf1    train_loss    valid_acc    valid_loss    cp       dur
-------  ---------  ---------  ------------  -----------  ------------  ----  --------
      1     0.9658     0.9913        0.0333       0.9913        0.0316     +  297.0346
      2     0.8987     0.9586        0.0201       0.9586        0.1227        298.8108
      3     0.9781     0.9940        0.0178       0.9940        0.0197     +  297.9007
      4     0.9807     0.9953        0.0161       0.9953        0.0160     +  298.7638
      5     0.9798     0.9948        0.0152       0.9948        0.0171        297.4060
      6     0.9829     0.9956        0.0142       0.9956        0.0148     +  297.7647
      7     0.9814     0.9952        0.0136       0.9952        0.0157        297.2848
      8     0.9829     0.9957        0.0130       0.9957        0.0142     +  299.7893
      9     0.9810     0.9954        0.0126       0.9954        0.0159        299.1556
     10     0.9823     0.9955        0.0120

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (d1): DownSample2x(
      (0): Conv1d(1, 64, kernel_size=(2,), stride=(2,))
      (1): ReLU(inplace=True)
    )
    (c1): ResConv1d(
      (cal): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool1d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): PReLU(num_parameters=1)
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): Sigmoid()
        )
      )
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (relu): ReLU()
 

In [9]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=16,
    lr=0.001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model23')],
    device='cuda'
)

net.fit(traval_tra_features, traval_tra_targets)

net = NeuralNetClassifier(
    MyModule,
    max_epochs=10,
    lr=0.0001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    criterion=F1Loss,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model23', fn_prefix='f1')],
    device='cuda'
)

net.initialize() 
net.load_params(f_params='03_model23/params.pt')
net.partial_fit(traval_tra_features, traval_tra_targets)

/home/amax/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


  epoch    macrof1    microf1    train_loss    valid_acc    valid_loss    cp       dur
-------  ---------  ---------  ------------  -----------  ------------  ----  --------
      1     0.9642     0.9884        0.0353       0.9884        0.0331     +  299.1565
      2     0.9736     0.9934        0.0210       0.9934        0.0222     +  295.6930
      3     0.9798     0.9950        0.0180       0.9950        0.0162     +  302.4762
      4     0.9794     0.9949        0.0164       0.9949        0.0167        302.3084
      5     0.9788     0.9942        0.0152       0.9942        0.0180        302.4575
      6     0.9813     0.9953        0.0145       0.9953        0.0157     +  304.5403
      7     0.9773     0.9943        0.0137       0.9943        0.0179        301.1611
      8     0.9830     0.9957        0.0131       0.9957        0.0145     +  302.4969
      9     0.9824     0.9957        0.0126       0.9957        0.0138        302.1068
     10     0.9815     0.9956        0.0122

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (d1): DownSample2x(
      (0): Conv1d(1, 64, kernel_size=(2,), stride=(2,))
      (1): ReLU(inplace=True)
    )
    (c1): ResConv1d(
      (cal): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool1d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): PReLU(num_parameters=1)
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): Sigmoid()
        )
      )
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (relu): ReLU()
 

In [10]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=16,
    lr=0.001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model24')],
    device='cuda'
)

net.fit(traval_tra_features, traval_tra_targets)

net = NeuralNetClassifier(
    MyModule,
    max_epochs=10,
    lr=0.0001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    criterion=F1Loss,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model24', fn_prefix='f1')],
    device='cuda'
)

net.initialize() 
net.load_params(f_params='03_model24/params.pt')
net.partial_fit(traval_tra_features, traval_tra_targets)

/home/amax/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


  epoch    macrof1    microf1    train_loss    valid_acc    valid_loss    cp       dur
-------  ---------  ---------  ------------  -----------  ------------  ----  --------
      1     0.9386     0.9795        0.0346       0.9795        0.0720     +  305.8630
      2     0.9658     0.9921        0.0202       0.9921        0.0247     +  306.2593
      3     0.9737     0.9932        0.0178       0.9932        0.0213     +  303.1106
      4     0.9803     0.9950        0.0161       0.9950        0.0163     +  306.3297
      5     0.9820     0.9956        0.0151       0.9956        0.0146     +  303.5151
      6     0.9826     0.9957        0.0144       0.9957        0.0143     +  302.1786
      7     0.9824     0.9955        0.0135       0.9955        0.0149        303.5883
      8     0.9786     0.9947        0.0131       0.9947        0.0158        303.9847
      9     0.9835     0.9959        0.0126       0.9959        0.0143     +  303.6690
     10     0.9833     0.9958        0.0120

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (d1): DownSample2x(
      (0): Conv1d(1, 64, kernel_size=(2,), stride=(2,))
      (1): ReLU(inplace=True)
    )
    (c1): ResConv1d(
      (cal): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool1d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): PReLU(num_parameters=1)
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): Sigmoid()
        )
      )
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (relu): ReLU()
 